In [1]:
import copy

import cufflinks
from plotly.offline import iplot, init_notebook_mode
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)
import pandas as pd
from kitedatatojson import kitejsondata, linearregression, normalization

In [2]:
ce=kitejsondata('11200ce.json')

In [3]:
data=ce[['h','l']]
data=data.join(linearregression(ce.h).iloc[:,0])
data=data.join(linearregression(ce.l).iloc[:,0])

In [4]:
data.fillna(0,inplace=True)
data=data.join(linearregression(data.slopeh10,5).iloc[:,0])
data=data.join(linearregression(data.slopel10,5).iloc[:,0])


In [5]:
fig=data[(data.index.day==29)&(data.index.hour>9)&(data.index.hour<13)].iplot(y=['h','l'],yTitle='11200CE'
            ,secondary_y=['slopeh10','slopel10','slopeslopeh105','slopeslopel105'],
            secondary_y_title='slopes',xTitle='Time',zerolinecolor='red',asFigure=True)
fig=fig.update_layout(xaxis_showgrid=False, yaxis_showgrid=False)
fig=fig.update_traces(mode='lines+markers',marker_size=3)

fig.show()

In [9]:
# amp=(data.slopeslopeh105-data.slopeslopel105).to_frame(name='difference')
# amp.difference=amp.difference.apply(normalization)
# amp=amp.difference[amp.difference != amp.difference.shift(1)].to_frame()
# amp=amp[amp.index.isin(ce.index)]
# amp=amp.join(data.slopeslopeh105)
# amp=amp.join(data.hlc)


In [7]:
# ce=ce[(ce.index.day==29)&(ce.index.hour>9)&(ce.index.hour<13)]
# amp=amp[amp.index.isin(ce.index)]
m=copy.deepcopy(fig)
# m=p.add_ohlc(x=ce.index,close=ce.c,high=ce.h,low=ce.l,open=ce.o,xaxis='x',yaxis='y',name='ohlc')
m.layout.xaxis.rangeselector.visible=False
m.layout.xaxis.rangeslider.visible=False

# m=m.add_scatter(x=amp.index,
#                 y=amp.slopeslopeh105,
#                 yaxis='y2',
#                 mode='markers',
#                 marker_line_color=amp.difference,
# #                 line_color='green',
#                 marker_size=15,
#                 marker_symbol='hash-dot',marker_line_width=1
#                )


# m=m.add_scatter(x=amp.index,
#                 y=amp.hlc,
#                 yaxis='y',
#                 mode='markers',
#                 marker_line_color=amp.difference,
# #                 line_color='red',
#                 marker_size=15,
#                 marker_symbol='x-thin',marker_line_width=1
#                )
m.layout.yaxis2.showgrid=False
m.layout.plot_bgcolor='black'
m.write_html('try_h_l.html')
m.show()

In [14]:
p=amp[['difference','hlc']]

In [17]:
p=p.join(p.shift(-1),lsuffix='_b',rsuffix='_s')

In [19]:
p=p[p.difference_b==1]

In [21]:
p.dropna(inplace=True)

In [23]:
p['profit']=p.hlc_s-p.hlc_b

In [25]:
p.to_excel('check_result.xlsx')